In [16]:
#load datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [37]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
def mae(ytrue, ypred): return mean_absolute_error(ytrue, ypred)
def mape(ytrue, ypred): return 100 * mean_absolute_percentage_error(ytrue, ypred)
def smape(ytrue, ypred): return 200 * np.mean(np.abs(ytrue - ypred) / (np.abs(ytrue) + np.abs(ypred)))
def rmse(ytrue, ypred): return math.sqrt(mean_squared_error(ytrue, ypred))
def rmae(ytrue, ypred, ynaive): return mean_absolute_error(ytrue, ypred) / mean_absolute_error(ytrue, ynaive)
def dae(ytrue, ypred):
    try:
        error = mae(ytrue.mean(axis=1), ypred.mean(axis=1))
    except:
        error = mae(ytrue, ypred)
    return error

In [59]:
df_predict.iloc[:, 0:24]

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,19.655796,16.169161,13.423910,7.460352,5.630969,5.655572,7.959844,11.575107,15.568828,19.784033,...,20.367432,18.770236,18.543219,28.155522,31.647609,32.996665,26.932367,21.742449,26.705800,25.556159
1,24.735270,21.370251,17.670811,11.501982,10.408439,16.915046,29.613868,41.070625,43.067180,42.519445,...,39.544023,36.652068,36.681371,45.434252,52.146235,52.113022,43.884947,36.511493,37.893144,36.036868
2,32.109005,29.905582,26.588791,20.542044,18.830926,23.978329,31.028982,40.014787,43.204747,43.976901,...,36.954285,34.652875,34.650613,42.796614,48.719101,47.389635,39.803978,34.320744,36.094757,33.771227
3,32.410985,30.155855,26.891733,21.234894,19.487566,24.424471,31.982029,41.468946,44.855420,46.243372,...,38.842942,36.574427,36.365148,43.388003,50.063705,48.815288,41.197438,35.091334,36.821150,33.978398
4,32.147931,29.584395,26.248439,18.712854,16.340612,21.844232,30.846546,39.723333,43.804707,43.854633,...,36.559569,34.358409,34.378164,42.074844,48.581817,46.872055,39.114707,32.645075,35.418451,33.575267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,39.344106,33.687740,27.472733,19.265377,17.537217,24.689487,37.498045,43.944420,44.785632,47.012832,...,46.242883,43.596699,44.576605,56.771747,62.450839,64.884429,56.894491,48.465698,50.698761,48.974877
717,45.160840,41.589210,36.749432,30.620955,30.135160,34.547641,40.298324,45.306646,47.547657,50.504653,...,46.688016,45.625408,47.681008,60.066504,65.050230,63.310922,55.581303,48.966859,51.070417,48.084849
718,43.045549,39.184533,35.152837,29.878301,28.764878,33.395263,40.229232,47.092810,49.947500,52.490577,...,48.255138,46.377247,48.016640,58.030645,63.776829,61.341643,54.298790,47.136554,49.180586,46.175856
719,43.038362,40.064127,35.764415,29.768331,28.049440,32.918710,38.228544,44.768260,46.927995,49.877547,...,45.980315,44.180801,45.776312,56.140904,62.715697,58.885380,53.317442,46.419814,48.337872,45.579741


In [111]:
os.chdir("c://Users//matis//OneDrive//Documents//ENSTA//3A//Capstone//project//tschora//EPFDAML")
from work.models.Feature import Naive
from work.models.Splitter import MySplitter
dataset = "2"

def load_naive_forecasts(dataset, country, labels):  
    naive_forecasts = {"train" : {}, "test" : {}, "validation": {}, "test_recalibrated" : {}}
    os.chdir("c://Users//matis//OneDrive//Documents//ENSTA//3A//Capstone//project//tschora//EPFDAML")

    if dataset in ("", "2", "3"): nval = 362
    else: nval = 365  
    spliter = MySplitter(nval, shuffle=False)
    model_wrapper = Naive("NAIVE", f"EPF{dataset}_{country}", labels) 
    # Need to re-split for taking the validation prices
    X, y = model_wrapper.load_train_dataset()
    ((Xtr, ytr), (Xv, yv)) = spliter(X, y)
    # Also computes the naive forecasts    
    naive_forecasts["validation"][country] = model_wrapper.predict(None, Xv) 
    naive_forecasts["train"][country] = model_wrapper.predict(None, Xtr)           
    Xt, yt = model_wrapper.load_test_dataset()
    naive_forecasts["test"][country] = model_wrapper.predict(None, Xt)
    naive_forecasts["test_recalibrated"][country] = model_wrapper.predict(None, Xt)  

    return naive_forecasts

In [122]:
country = "FRBE"
version = "2"


for country in ["FRBE", "FRDE", "DEBE", "FRDEBE"]:
    print("country", country)
    os.chdir("c://Users//matis//OneDrive//Documents//ENSTA//3A//Capstone//project//tschora//EPFDAML//data")
    path_predict = f"datasets\EPF{version}_{country}\MLP_TSCHORA_EPF{version}_{country}_test_predictions.csv"
    df_predict = pd.read_csv(path_predict, sep=",", index_col=None)
    path_true = f"datasets/EPF{version}_{country}/test.csv"
    df_true = pd.read_csv(path_true, sep=",", index_col=None)

    if len(country)==4:
        countries = [country[:2], country[2:]]
    elif len(country)==6:
        countries = [country[:2], country[2:4], country[4:]]
    else:  countries = [country]

    cols_price = []
    for c in countries:
        cols_price += [c+"_price_"+str(i) for i in range(24)]
    
    naive = load_naive_forecasts(version, country, cols_price)


    # generalize metrics (on all countries)
    ytrue = df_true[cols_price].values.reshape(1,-1)
    ypred = df_predict.values.reshape(1,-1)
    ynaive = naive["test"][country].reshape(1,-1)
    print(ytrue.shape, ypred.shape, ynaive.shape)
    mae_val,mape_val, smape_val, rmse_val, rmae_val, dae_val = mae(ytrue, ypred), mape(ytrue, ypred), smape(ytrue, ypred), rmse(ytrue, ypred), rmae(ytrue, ypred, ynaive), dae(ytrue, ypred)

    # single metrics (on single country)
    mae_list, mape_list, smape_list, rmse_list, rmae_list, dae_list = [], [], [], [], [], []
    for i, c in enumerate(countries):
        cols_c = [c+"_price_"+str(i) for i in range(24)]
        ytrue = df_true[cols_c].values.reshape(1,-1)
        ypred = df_predict.iloc[:, 24*i:24*(i+1)].values.reshape(1,-1)
        ynaive = naive["test"][country][:, 24*i:24*(i+1)].reshape(1,-1)
        mae_list.append(mae(ytrue, ypred))
        mape_list.append(mape(ytrue, ypred))
        smape_list.append(smape(ytrue, ypred))
        rmse_list.append(rmse(ytrue, ypred))
        rmae_list.append(rmae(ytrue, ypred, ynaive))
        dae_list.append(dae(ytrue, ypred))

    # print results in dataframe
    res_gen = pd.DataFrame({"MAE": mae_val, "MAPE": mape_val, "SMAPE": smape_val, "RMSE": rmse_val, "RMAE": rmae_val, "DAE": dae_val}, index=["General"])
    res_uni = pd.DataFrame({"MAE": mae_list, "MAPE": mape_list, "SMAPE": smape_list, "RMSE": rmse_list, "RMAE": rmae_list, "DAE": dae_list}, index=countries)
    res = pd.concat([res_gen, res_uni])
    print(f"Results for {country} - EPF{version} - MLP_TSCHORA_EPF{version}_{country}")
    print(res)
        





country FRBE
(1, 34608) (1, 34608) (1, 34608)
Results for FRBE - EPF2 - MLP_TSCHORA_EPF2_FRBE
              MAE       MAPE      SMAPE       RMSE      RMAE       DAE
General  5.721823  21.957466  15.008659  14.882179  0.736975  2.136997
FR       4.791156  15.025672  13.934142  12.382026  0.714893  1.602307
BE       6.652491  28.889260  16.083176  17.018929  0.753742  2.671687
country FRDE
(1, 34608) (1, 34608) (1, 34608)
Results for FRDE - EPF2 - MLP_TSCHORA_EPF2_FRDE
              MAE          MAPE      SMAPE       RMSE      RMAE       DAE
General  8.739397  5.741792e+14  40.545907  13.631276  1.251494  5.007696
FR       8.927647  3.489911e+01  36.016540  15.854817  1.332103  5.327332
DE       8.551148  1.148358e+15  45.075274  10.965771  1.177127  4.688060
country DEBE
(1, 34608) (1, 34608) (1, 34608)
Results for DEBE - EPF2 - MLP_TSCHORA_EPF2_DEBE
              MAE          MAPE      SMAPE       RMSE      RMAE       DAE
General  5.677762  1.082490e+14  18.201424  13.010289  0.705734 

In [120]:
naive["test"][country].shape

(721, 48)

In [96]:
cols_price

['FR_price_0',
 'FR_price_1',
 'FR_price_2',
 'FR_price_3',
 'FR_price_4',
 'FR_price_5',
 'FR_price_6',
 'FR_price_7',
 'FR_price_8',
 'FR_price_9',
 'FR_price_10',
 'FR_price_11',
 'FR_price_12',
 'FR_price_13',
 'FR_price_14',
 'FR_price_15',
 'FR_price_16',
 'FR_price_17',
 'FR_price_18',
 'FR_price_19',
 'FR_price_20',
 'FR_price_21',
 'FR_price_22',
 'FR_price_23',
 'DE_price_0',
 'DE_price_1',
 'DE_price_2',
 'DE_price_3',
 'DE_price_4',
 'DE_price_5',
 'DE_price_6',
 'DE_price_7',
 'DE_price_8',
 'DE_price_9',
 'DE_price_10',
 'DE_price_11',
 'DE_price_12',
 'DE_price_13',
 'DE_price_14',
 'DE_price_15',
 'DE_price_16',
 'DE_price_17',
 'DE_price_18',
 'DE_price_19',
 'DE_price_20',
 'DE_price_21',
 'DE_price_22',
 'DE_price_23',
 'BE_price_0',
 'BE_price_1',
 'BE_price_2',
 'BE_price_3',
 'BE_price_4',
 'BE_price_5',
 'BE_price_6',
 'BE_price_7',
 'BE_price_8',
 'BE_price_9',
 'BE_price_10',
 'BE_price_11',
 'BE_price_12',
 'BE_price_13',
 'BE_price_14',
 'BE_price_15',
 'BE_p

In [104]:
naive = load_naive_forecasts(dataset, country, cols_price)

In [106]:
naive["test"][country].reshape(1,-1).shape

(1, 51912)